In [1]:
import torch
# utilizado para la manipulación de directorios y rutas
import os
import pandas as pd
# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [2]:
D_in, H, D_out = 784, 55, 26

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

In [3]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 26])

In [4]:
model.to("cuda")


Sequential(
  (0): Linear(in_features=784, out_features=55, bias=True)
  (1): ReLU()
  (2): Linear(in_features=55, out_features=26, bias=True)
)

In [5]:
import pandas as pd

def csvAdapted(dataset, modifyColumns, dropColumns):
    data = pd.read_csv(dataset, encoding='latin-1')
    columns = data.columns

    for index in modifyColumns:
        name = columns[index]
        value = list( data[name].unique() )
        data[name] = data[name].map( dict( zip( value,  [i for i in range( len(value) ) ] ) ) )

    data = data.drop([columns[index] for index in dropColumns ], axis=1)
    data = data.dropna()
#data= data.drop('nombre', axis=1) 1 elimina una columna, 0 elimina una fila
    print(data.head()) #para imprimir el dataset

    return data.to_numpy()

data = csvAdapted("/content/digital_letters.csv", [785],[0]) #cambia la columna 785 y les asigna valores, elimina la columna 0

     0    1    2    3    4    5    6    7    8    9  ...  775  776  777  778  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

   779  780  781  782  783  label  
0  0.0  0.0  0.0  0.0  0.0      0  
1  0.0  0.0  0.0  0.0  0.0      1  
2  0.0  0.0  0.0  0.0  0.0      2  
3  0.0  0.0  0.0  0.0  0.0      3  
4  0.0  0.0  0.0  0.0  0.0      4  

[5 rows x 785 columns]


In [6]:

# print(data)
X, Y = data[:, :784], data[:, 784]
print(X.shape)
print(Y.shape)

(13130, 784)
(13130,)


In [9]:
import numpy as np

# normalización y split

X_train, X_test, y_train, y_test = X[:10000] / 255., X[10000:] / 255., Y[:10000].astype(np.int), Y[10000:].astype(np.int)

<ipython-input-9-9a888fb6e7a8>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train, X_test, y_train, y_test = X[:10000] / 255., X[10000:] / 255., Y[:10000].astype(np.int), Y[10000:].astype(np.int)


In [10]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [11]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 100
lr = 0.6
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 3.11526
Epoch 20/100 Loss 2.83092
Epoch 30/100 Loss 2.58649
Epoch 40/100 Loss 2.41727
Epoch 50/100 Loss 2.27265
Epoch 60/100 Loss 2.16306
Epoch 70/100 Loss 2.07278
Epoch 80/100 Loss 1.99536
Epoch 90/100 Loss 1.92766
Epoch 100/100 Loss 1.86810


In [12]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.6217252396166134